In [30]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter


# Application of Bag of Words Approach to Author Identification

In [31]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [32]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [33]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents[:500] + persuasion_sents[:500])
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


Time to bag some words!  Since spaCy has already tokenized and labelled our data, we can move directly to recording how often various words occur.  We will exclude stopwords and punctuation.  In addition, in an attempt to keep our feature space from exploding, we will work with lemmas (root words) rather than the raw text terms, and we'll only use the 2000 most common words for each text.

In [35]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [36]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500


,gently,thousand,operation,give,action,manage,edition,remembrance,frequent,fourth,...,flamingo,real,humour,race,wish,worth,languid,double,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


## Trying out BoW

Now let's give the bag of words features a whirl by trying a random forest.

In [41]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier(n_estimators = 2000, max_depth = 13)
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.9116666666666666

Test set score: 0.8775



## BoW with Logistic Regression


In [56]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=1e-1)
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(600, 3062) (600,)
Training set score: 0.9516666666666667

Test set score: 0.895


# Lasso

In [66]:
lasso = LogisticRegression(penalty='l1', C=1e1)
train = lasso.fit(X_train, y_train)
print(X_train.shape, y_train.shape)

print('Training set score:', lasso.score(X_train, y_train))
print('\nTest set score:', lasso.score(X_test, y_test))

(600, 3062) (600,)
Training set score: 0.9883333333333333

Test set score: 0.85



# BoW with Gradient Boosting

In [44]:
clf = ensemble.GradientBoostingClassifier(n_estimators=700, max_depth = 5, learning_rate=0.01)
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.93

Test set score: 0.835


# SVC

In [74]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
svc = SVC(C=1e3, kernel = 'rbf', gamma = 0.001)

np.mean(cross_val_score(svc, X_train, y_train, n_jobs= -1))

0.84

In [75]:
svc.fit(X_train, y_train)
print('\nTest set score:', svc.score(X_test, y_test))


Test set score: 0.8425


# XGBoost

In [97]:
from xgboost import XGBClassifier
n_est = []
scores = []

for value in [100, 500, 700, 1000]:
    xgb = XGBClassifier(n_jobs = -1,  n_estimators = value, booster= 'gbtree')
    score = np.mean(cross_val_score(xgb, X_train, y_train, n_jobs=-1))
    scores.append(score)
    n_est.append(value)

In [98]:
df_xgb = pd.DataFrame(scores, columns = ['scores'])
df_xgb['n_est'] = n_est
df_xgb.sort_values(by=['scores'], ascending=False)

,scores,n_est
2,0.811667,700
1,0.810000,500
3,0.808333,1000
0,0.773333,100


In [100]:
max_depth = []
scores = []

for value in [2, 4, 6, 8]:
    xgb = XGBClassifier(n_jobs = -1,  max_depth = value, n_estimators = 700,booster= 'gbtree')
    score = np.mean(cross_val_score(xgb, X_train, y_train, n_jobs=-1))
    scores.append(score)
    max_depth.append(value)

In [101]:
df_xgb = pd.DataFrame(scores, columns = ['scores'])
df_xgb['max_depth'] = max_depth
df_xgb.sort_values(by=['scores'], ascending=False)

,scores,max_depth
2,0.808333,6
1,0.806667,4
3,0.806667,8
0,0.803333,2


In [103]:
learning_rates = []
scores = []

for value in [0.5, 0.1, 0.01]:
    xgb = XGBClassifier(n_jobs = -1,  max_depth = 6, n_estimators = 700,booster= 'gbtree', learning_rate=value)
    score = np.mean(cross_val_score(xgb, X_train, y_train, n_jobs=-1))
    scores.append(score)
    learning_rates.append(value)

In [104]:
df_xgb = pd.DataFrame(scores, columns = ['scores'])
df_xgb['learning_rate'] = learning_rates
df_xgb.sort_values(by=['scores'], ascending=False)

,scores,learning_rate
0,0.810000,0.50
1,0.808333,0.10
2,0.773333,0.01


In [106]:
xgb = XGBClassifier(n_jobs = -1,  max_depth = 6, n_estimators = 700,booster= 'gbtree', learning_rate=0.5)

In [107]:
xgb.fit(X_train, y_train)
print('\nTest set score:', xgb.score(X_test, y_test))


Test set score: 0.835


# Same Model, New Text

What if we use a different text by Austen, e.g. *Emma*?

In [76]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [77]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [78]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

# Emma is quite long, let's cut it down to the same length as Alice.
emma_sents = emma_sents[0:500]

In [79]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Processing row 0
done


In [80]:
# Now we can model it!

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])



Test set score: 0.6146095717884131


col_0,Austen,Carroll
row_0,,
Austen,332,168
Carroll,138,156


## Logistic Regression on *Emma*

In [81]:
# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.6146095717884131


col_0,Austen,Carroll
row_0,,
Austen,332,168
Carroll,138,156


## Lasso

In [84]:
print('\nTest set score:', lasso.score(X_Emma_test, y_Emma_test))


Test set score: 0.5818639798488665


## Random Forest Classifier

In [85]:
print('\nTest set score:', rfc.score(X_Emma_test, y_Emma_test))


Test set score: 0.5390428211586902


## Gradient Boosting

In [86]:
print('\nTest set score:', clf.score(X_Emma_test, y_Emma_test))


Test set score: 0.7015113350125944


## SVC

In [87]:
print('\nTest set score:', svc.score(X_Emma_test, y_Emma_test))


Test set score: 0.5818639798488665


# XGBoost

In [108]:
print('\nTest set score:', xgb.score(X_Emma_test, y_Emma_test))


Test set score: 0.6007556675062973


# How about Another Text by Austen?


In [122]:
# Clean the Hamlet data.
sense = gutenberg.raw('austen-sense.txt')
sense_cleaned = text_cleaner(sense)

In [123]:
sense_doc = nlp(sense_cleaned )

In [124]:
sense_sents = [[sent, 'Austen'] for sent in sense_doc.sents]
sense_sents = sense_sents[0:500]

In [125]:
sense_sentences =pd.DataFrame(sense_sents)
sense_bow = bow_features(sense_sentences, common_words)

Processing row 0


In [127]:
X_sense_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    sense_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_sense_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * sense_bow.shape[0])])

# Testing Models on *Sense and Sensibility*

In [128]:
print('\nTest set score:', lr.score(X_sense_test, y_sense_test))


Test set score: 0.6675062972292192


In [129]:
print('\nTest set score:', lasso.score(X_sense_test, y_sense_test))


Test set score: 0.6322418136020151


In [130]:
print('\nTest set score:', rfc.score(X_sense_test, y_sense_test))


Test set score: 0.5919395465994962


In [132]:
print('\nTest set score:', svc.score(X_sense_test, y_sense_test))


Test set score: 0.6209068010075567


In [133]:
print('\nTest set score:', xgb.score(X_sense_test, y_sense_test))


Test set score: 0.6297229219143576


## Out of all models, Gradient Boosting Classifier performed the best on the new text

In [131]:
print('\nTest set score:', clf.score(X_sense_test, y_sense_test))


Test set score: 0.7115869017632241
